In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.lines import Line2D # For custom legends if needed later
from tqdm import tqdm

# --- Matplotlib Styling (Optional) ---
plt.rcParams.update({
    'font.size': 10,
    'axes.titlesize': 13, # Slightly smaller for subplots
    'axes.labelsize': 11,
    'xtick.labelsize': 8, # Smaller for potentially many layer names
    'ytick.labelsize': 9,
    'legend.fontsize': 8,
    'figure.titlesize': 15, # Main figure title
    'lines.linewidth': 1.5,
    'lines.markersize': 4
})

# --- 1. Helper Functions ---

def compute_effective_rank(activations_batch):
    if activations_batch.ndim == 1:
        if activations_batch.shape[0] > 1: return 1.0
        else: activations_batch = activations_batch.unsqueeze(0)
    if activations_batch.shape[1] == 0: return 0.0
    if activations_batch.shape[1] == 1: return 1.0
    if activations_batch.shape[0] <= 1: return 1.0

    std_devs = torch.std(activations_batch, dim=0)
    valid_features_mask = std_devs > 1e-5
    if valid_features_mask.sum() < 2: return float(valid_features_mask.sum().item())
    activations_batch_filtered = activations_batch[:, valid_features_mask]

    try:
        if activations_batch_filtered.shape[0] <= 1 or activations_batch_filtered.shape[1] < 2:
            return float(activations_batch_filtered.shape[1] > 0)
        corr_matrix = torch.corrcoef(activations_batch_filtered.T)
    except RuntimeError: return 1.0
    if torch.isnan(corr_matrix).any(): corr_matrix = torch.nan_to_num(corr_matrix, nan=0.0)

    s_unnormalized = torch.linalg.svdvals(corr_matrix)
    sum_s = torch.sum(s_unnormalized)
    if sum_s < 1e-12: return 0.0
    s_norm_for_entropy = s_unnormalized / sum_s
    s_norm_for_entropy = s_norm_for_entropy[s_norm_for_entropy > 1e-15]
    if len(s_norm_for_entropy) == 0: return 0.0
    entropy = -torch.sum(s_norm_for_entropy * torch.log(s_norm_for_entropy))
    return torch.exp(entropy).item()

def get_activation_fn_and_name(activation_name_str):
    name = activation_name_str.lower()
    if name == "relu": return nn.ReLU(), "ReLU"
    elif name == "tanh": return nn.Tanh(), "Tanh"
    elif name == "sigmoid": return nn.Sigmoid(), "Sigmoid"
    elif name == "gelu": return nn.GELU(), "GELU"
    elif name == "identity": return nn.Identity(), "Identity"
    else: raise ValueError(f"Unsupported activation: {activation_name_str}")

# --- 2. Model Definition ---
class MLPBlock(nn.Module):
    def __init__(self, in_dim, out_dim, norm_type, activation_fn_instance, dropout_p, is_output_layer_block, block_idx):
        super().__init__()
        self.linear = nn.Linear(in_dim, out_dim)
        self.norm = None
        self.act_fn_instance = activation_fn_instance # Store for naming
        self.norm_type_str = norm_type # Store for naming
        self.is_output_layer_block = is_output_layer_block
        self.block_idx = block_idx # e.g., 0 for H1, 1 for H2...

        layer_prefix = f"H{block_idx + 1}" if not is_output_layer_block else "Output"
        self.layer_names = {"linear": f"{layer_prefix}_Linear"}


        if not is_output_layer_block and norm_type:
            if norm_type.lower() == "batchnorm":
                self.norm = nn.BatchNorm1d(out_dim)
                self.layer_names["norm"] = f"{layer_prefix}_BatchNorm"
            elif norm_type.lower() == "layernorm":
                self.norm = nn.LayerNorm(out_dim)
                self.layer_names["norm"] = f"{layer_prefix}_LayerNorm"
            elif norm_type.lower() != "none":
                raise ValueError(f"Unsupported norm_type: {norm_type}")
        
        if not is_output_layer_block:
            self.act = activation_fn_instance
            act_name = self.act.__class__.__name__ if not isinstance(self.act, nn.modules.activation.Tanh) else "Tanh" # Tanh doesn't have a clean name sometimes
            if isinstance(self.act, nn.Identity): act_name = "Identity" # Handle explicit Identity
            self.layer_names["act"] = f"{layer_prefix}_{act_name}"
        else:
            self.act = nn.Identity() # Output layer has no activation for logits

        self.dropout = nn.Dropout(dropout_p) if not is_output_layer_block and dropout_p > 0 else nn.Identity()

    def forward(self, x, record_activations_in_block=False):
        recorded_stages_in_block = [] # List of (name, tensor)

        x_linear = self.linear(x)
        if record_activations_in_block:
            recorded_stages_in_block.append((self.layer_names["linear"], x_linear.detach().cpu()))
        
        current_x = x_linear
        if not self.is_output_layer_block:
            if self.norm:
                current_x = self.norm(current_x)
                if record_activations_in_block:
                    recorded_stages_in_block.append((self.layer_names["norm"], current_x.detach().cpu()))
            
            current_x = self.act(current_x)
            if record_activations_in_block:
                recorded_stages_in_block.append((self.layer_names["act"], current_x.detach().cpu()))
            
            output_for_next_layer = self.dropout(current_x)
        else: # Output layer block
            output_for_next_layer = current_x # This is the logits

        if record_activations_in_block:
            return output_for_next_layer, recorded_stages_in_block
        else:
            return output_for_next_layer, []


class ConfigurableMLP(nn.Module):
    def __init__(self, layer_dims, activation_name="relu", norm_type=None, dropout_p=0.0):
        super().__init__()
        self.layer_dims = layer_dims
        self.input_dim = layer_dims[0]
        self.activation_name_str = activation_name # Store for reference
        self.norm_type_str = norm_type
        
        act_fn_instance, _ = get_activation_fn_and_name(activation_name)
        self.layer_stage_names = ["Input"] # Start with input

        self.blocks = nn.ModuleList()
        for i in range(len(layer_dims) - 1):
            is_final_block = (i == len(layer_dims) - 2)
            block = MLPBlock(
                layer_dims[i], layer_dims[i+1],
                norm_type, act_fn_instance, dropout_p,
                is_output_layer_block=is_final_block,
                block_idx=i 
            )
            self.blocks.append(block)
            # Collect layer names from the block
            self.layer_stage_names.append(block.layer_names["linear"])
            if "norm" in block.layer_names:
                self.layer_stage_names.append(block.layer_names["norm"])
            if "act" in block.layer_names: # "act" only if not output block
                 self.layer_stage_names.append(block.layer_names["act"])
            
    def forward(self, x, record_activations=False):
        x_flattened = x.view(x.size(0), -1)
        if x_flattened.shape[1] != self.input_dim:
            raise ValueError(f"Input dim mismatch: {x_flattened.shape[1]} vs {self.input_dim}")

        all_recorded_stages = [] # List of (name, tensor)
        current_features = x_flattened
        
        if record_activations:
            all_recorded_stages.append(("Input", current_features.detach().cpu()))

        for block in self.blocks:
            current_features, block_recorded_activations = block(current_features, record_activations_in_block=record_activations)
            if record_activations:
                all_recorded_stages.extend(block_recorded_activations)
        
        final_model_output = current_features

        if record_activations:
            return final_model_output, all_recorded_stages
        return final_model_output

# --- 3. Data Handling ---
def get_cifar10_dataloaders(batch_size=128, num_workers=2):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
    try:
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform) # Changed path
    except Exception as e: print(f"Error training set: {e}"); raise
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=torch.cuda.is_available())
    try:
        testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform) # Changed path
    except Exception as e: print(f"Error test set: {e}"); raise
    testloader = DataLoader(testset, batch_size=batch_size*2, shuffle=False, num_workers=num_workers, pin_memory=torch.cuda.is_available())
    return trainloader, testloader

# --- 4. Training and Evaluation Loop with Rank Logging ---
def _log_ranks_for_epoch(model, fixed_val_images, epoch_num, model_config_name, all_rank_data_list):
    model.eval()
    with torch.no_grad():
        _, recorded_stages = model(fixed_val_images, record_activations=True)
        current_epoch_ranks_info = []
        for stage_name, acts_batch in recorded_stages:
            eff_rank = np.nan
            if acts_batch.size(0) > 1 and acts_batch.ndim > 1 and acts_batch.shape[1] > 0:
                eff_rank = compute_effective_rank(acts_batch)
            
            all_rank_data_list.append({
                'model_config': model_config_name, 'epoch': epoch_num,
                'layer_name': stage_name, 'eff_rank': eff_rank
            })
            current_epoch_ranks_info.append(f"{stage_name}:{eff_rank:.1f}" if not np.isnan(eff_rank) else f"{stage_name}:NaN")
    
    print(f"Epoch {epoch_num} [{model_config_name}] Ranks: {' | '.join(current_epoch_ranks_info[:5])} ...") # Print first few
    model.train()


def train_model_with_rank_logging(
    model_config_name, model, train_loader, val_loader, 
    optimizer, criterion, num_epochs, device, 
    log_rank_every_n_epochs=1, rank_eval_batch_size=256
    ):
    
    all_rank_data = [] 
    fixed_val_images = None 
    try: # Prepare fixed validation batch
        fixed_val_images_list = []
        num_batches_for_rank = 1
        if val_loader.batch_size and val_loader.batch_size > 0 :
            num_batches_for_rank = max(1, rank_eval_batch_size // val_loader.batch_size)
        elif rank_eval_batch_size > 0 :
             print(f"Warning: val_loader.batch_size is None/0. Will fetch {rank_eval_batch_size} samples.")
        else: raise ValueError("val_loader.batch_size is None/0, and rank_eval_batch_size not positive.")

        val_iter_for_rank = iter(val_loader)
        current_fetched_samples = 0
        while current_fetched_samples < rank_eval_batch_size:
            try:
                imgs, _ = next(val_iter_for_rank)
                fixed_val_images_list.append(imgs)
                current_fetched_samples += imgs.size(0)
            except StopIteration: print("Warning: Val loader exhausted for rank batch."); break
        if not fixed_val_images_list: raise ValueError("Val loader empty/too small for rank batch.")
        fixed_val_images_cat = torch.cat(fixed_val_images_list, dim=0)
        if fixed_val_images_cat.size(0) > rank_eval_batch_size:
            fixed_val_images_cat = fixed_val_images_cat[:rank_eval_batch_size]
        if fixed_val_images_cat.size(0) == 0: raise ValueError("No images for fixed_val_images.")
        fixed_val_images = fixed_val_images_cat.to(device)
        print(f"Using fixed val batch of size {fixed_val_images.shape[0]} for rank eval.")
    except Exception as e:
        print(f"Could not prepare fixed val batch: {e}. Falling back to train batch.")
        try:
            fb_imgs, _ = next(iter(train_loader)) 
            if fb_imgs.size(0) > rank_eval_batch_size: fb_imgs = fb_imgs[:rank_eval_batch_size]
            fixed_val_images = fb_imgs.to(device)
        except Exception as fallback_e: print(f"Fallback failed: {fallback_e}. No rank logging."); return pd.DataFrame(all_rank_data)

    # --- Log Ranks at Epoch 0 (Initialization) ---
    if fixed_val_images is not None and fixed_val_images.nelement() > 0:
        print(f"Logging ranks for Epoch 0 (Initialization) for {model_config_name}...")
        _log_ranks_for_epoch(model, fixed_val_images, 0, model_config_name, all_rank_data)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), 
                            desc=f"Epoch {epoch+1}/{num_epochs} [{model_config_name}]", leave=False)
        for i, data in progress_bar:
            inputs, labels = data; inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad(); outputs = model(inputs); loss = criterion(outputs, labels)
            loss.backward(); optimizer.step()
            running_loss += loss.item()
            if (i + 1) % 100 == 0: progress_bar.set_postfix({'loss': f'{running_loss / 100:.3f}'}); running_loss = 0.0
        
        if (epoch + 1) % log_rank_every_n_epochs == 0 and fixed_val_images is not None and fixed_val_images.nelement() > 0:
            _log_ranks_for_epoch(model, fixed_val_images, epoch + 1, model_config_name, all_rank_data)

    print(f"Finished training for {model_config_name}")
    return pd.DataFrame(all_rank_data)

# --- 5. Main Experiment Orchestration ---
def run_main_experiment(
    num_epochs=10, log_rank_every_n_epochs=1, mlp_hidden_layers=[256, 128], 
    activation_name="relu", dropout_p=0.1, learning_rate=1e-3,
    batch_size=256, rank_eval_batch_size_config=512
    ):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"); print(f"Using device: {device}")
    try: train_loader, val_loader = get_cifar10_dataloaders(batch_size=batch_size)
    except Exception as e: print(f"Failed to load CIFAR-10: {e}. Aborting."); return

    input_dim = 32 * 32 * 3; num_classes = 10
    layer_dims_template = [input_dim] + mlp_hidden_layers + [num_classes]

    configurations = {
        "MLP NoNorm": {"norm_type": None},
        "MLP BatchNorm": {"norm_type": "batchnorm"},
        "MLP LayerNorm": {"norm_type": "layernorm"}
    }
    all_experimental_rank_data = []
    # To get consistent layer stage names for plotting x-axis later
    sample_model_for_layer_names = ConfigurableMLP(layer_dims_template, activation_name, None, 0.0)
    plot_layer_stage_names = sample_model_for_layer_names.layer_stage_names
    del sample_model_for_layer_names


    for config_name, params in configurations.items():
        print(f"\n--- Running Configuration: {config_name} ---")
        model = ConfigurableMLP(layer_dims_template, activation_name, params["norm_type"], dropout_p).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate); criterion = nn.CrossEntropyLoss()
        rank_data_df = train_model_with_rank_logging(
            config_name, model, train_loader, val_loader, optimizer, criterion, 
            num_epochs, device, log_rank_every_n_epochs, rank_eval_batch_size_config
        )
        all_experimental_rank_data.append(rank_data_df)
        del model; 
        if torch.cuda.is_available(): torch.cuda.empty_cache()

    if not all_experimental_rank_data or all(df.empty for df in all_experimental_rank_data):
        print("No rank data. Exiting plot."); return
    final_rank_df = pd.concat(all_experimental_rank_data, ignore_index=True)

    # --- 6. Visualization ---
    if final_rank_df.empty: print("Final rank df empty. Skipping plot."); return
        
    num_configs = len(configurations)
    fig, axs = plt.subplots(num_configs, 1, figsize=(12, 6 * num_configs), sharex=True, sharey=True)
    if num_configs == 1: axs = [axs] # Ensure axs is always a list

    unique_epochs = sorted(final_rank_df['epoch'].dropna().unique())
    if not unique_epochs: print("No epochs in data. Skipping plot."); return
    
    epochs_to_plot = unique_epochs
    if len(unique_epochs) > 10: # Select a subset if too many epochs for clarity
        indices = np.round(np.linspace(0, len(unique_epochs) - 1, min(10, len(unique_epochs)))).astype(int)
        epochs_to_plot = [unique_epochs[i] for i in indices]

    epoch_cmap = plt.cm.get_cmap('viridis', len(epochs_to_plot))
    
    # Map layer names to integer indices for consistent plotting on x-axis
    # Use the predefined plot_layer_stage_names derived from a sample model
    layer_name_to_idx = {name: i for i, name in enumerate(plot_layer_stage_names)}


    for i, (config_name, params_config) in enumerate(configurations.items()):
        ax = axs[i]
        config_data = final_rank_df[final_rank_df['model_config'] == config_name]
        if config_data.empty: ax.set_title(f"{config_name} (No Data)"); continue

        for epoch_idx, epoch in enumerate(epochs_to_plot):
            epoch_specific_data = config_data[config_data['epoch'] == epoch].copy()
            if not epoch_specific_data.empty:
                # Map layer names to their pre-defined indices for x-axis
                epoch_specific_data['plot_x_idx'] = epoch_specific_data['layer_name'].map(layer_name_to_idx)
                epoch_specific_data.sort_values(by='plot_x_idx', inplace=True)
                
                # Filter out rows where plot_x_idx might be NaN (if a layer_name was unexpected)
                epoch_specific_data_plot = epoch_specific_data.dropna(subset=['plot_x_idx', 'eff_rank'])

                if not epoch_specific_data_plot.empty:
                    ax.plot(epoch_specific_data_plot['plot_x_idx'], epoch_specific_data_plot['eff_rank'],
                            color=epoch_cmap(epoch_idx), 
                            linestyle='-', marker='o', alpha=0.7, markersize=5,
                            label=f"Epoch {epoch}" if i == 0 else None # Label epochs only on the first subplot's lines
                           )
        
        ax.set_title(f"Effective Rank: {config_name} (Activation: {activation_name.upper()})")
        ax.set_ylabel("Effective Rank")
        ax.grid(True, linestyle='--', alpha=0.6)
        if i == num_configs - 1: # X-axis labels only on the last subplot
            ax.set_xticks(list(layer_name_to_idx.values()))
            ax.set_xticklabels([name.replace("_", " ") for name in plot_layer_stage_names], rotation=45, ha='right')
            ax.set_xlabel("Layer Stage")
        else:
            ax.set_xticklabels([]) # No x-labels for upper subplots

    # Common colorbar for epochs
    norm_epochs = plt.Normalize(vmin=min(epochs_to_plot), vmax=max(epochs_to_plot))
    sm = plt.cm.ScalarMappable(cmap=epoch_cmap, norm=norm_epochs)
    sm.set_array([])
    
    # Position colorbar to the right of all subplots
    fig.subplots_adjust(right=0.85) # Make space for colorbar
    cbar_ax = fig.add_axes([0.88, 0.15, 0.03, 0.7]) # [left, bottom, width, height]
    cbar = fig.colorbar(sm, cax=cbar_ax, orientation='vertical', label='Epoch')
    if epochs_to_plot:
        cbar.set_ticks(epochs_to_plot)
        cbar.ax.set_yticklabels([str(e) for e in epochs_to_plot])
    
    fig.tight_layout(rect=[0, 0, 0.85, 0.96]) # Adjust rect to prevent overlap with colorbar and title
    plt.suptitle(f"Effective Rank Evolution Across Layers and Epochs", y=0.99)
    plt.show()

if __name__ == "__main__":
    run_main_experiment(
        num_epochs=15,            
        log_rank_every_n_epochs=1, 
        mlp_hidden_layers=[128, 64], # Smaller for quicker test
        activation_name="relu",   
        dropout_p=0.0, # No dropout for clearer rank signal initially
        learning_rate=1e-3,
        batch_size=256,
        rank_eval_batch_size_config=512 
    )


Using device: cuda:0



--- Running Configuration: MLP NoNorm ---
Using fixed val batch of size 512 for rank eval.
Logging ranks for Epoch 0 (Initialization) for MLP NoNorm...
Epoch 0 [MLP NoNorm] Ranks: Input:34.7 | H1_Linear:30.4 | H1_ReLU:48.5 | H2_Linear:26.3 | H2_ReLU:38.0 ...


Epoch 1 [MLP NoNorm] Ranks: Input:34.7 | H1_Linear:14.4 | H1_ReLU:31.3 | H2_Linear:16.1 | H2_ReLU:28.2 ...


Epoch 2 [MLP NoNorm] Ranks: Input:34.7 | H1_Linear:18.3 | H1_ReLU:38.3 | H2_Linear:18.6 | H2_ReLU:31.4 ...


Epoch 3 [MLP NoNorm] Ranks: Input:34.7 | H1_Linear:21.2 | H1_ReLU:42.8 | H2_Linear:20.2 | H2_ReLU:32.9 ...


...

Epoch 13 [MLP NoNorm] Ranks: Input:34.7 | H1_Linear:32.6 | H1_ReLU:56.6 | H2_Linear:24.6 | H2_ReLU:40.5 ...


Epoch 14 [MLP NoNorm] Ranks: Input:34.7 | H1_Linear:33.6 | H1_ReLU:57.6 | H2_Linear:25.1 | H2_ReLU:41.0 ...


Epoch 15 [MLP NoNorm] Ranks: Input:34.7 | H1_Linear:33.0 | H1_ReLU:56.6 | H2_Linear:24.1 | H2_ReLU:40.4 ...
Finished training for MLP NoNorm

--- Running Configuration: MLP BatchNorm ---
Using fixed val batch of size 512 for rank eval.
Logging ranks for Epoch 0 (Initialization) for MLP BatchNorm...
Epoch 0 [MLP BatchNorm] Ranks: Input:34.7 | H1_Linear:29.7 | H1_BatchNorm:29.7 | H1_ReLU:47.7 | H2_Linear:27.8 ...


Epoch 1 [MLP BatchNorm] Ranks: Input:34.7 | H1_Linear:11.6 | H1_BatchNorm:11.6 | H1_ReLU:23.9 | H2_Linear:14.0 ...


Epoch 2/15 [MLP BatchNorm]:  81%|████████  | 158/196 [00:01<00:00, 84.69it/s, loss=1.457]